In [1]:
import docx
import os
from document_formatter_config import DocumentFormatterConfig
from document_formatting_agent import DocumentFormattingAgent

In [2]:
MAIN_DIR = "C:/Users/kaszt/OneDrive/Dokumenty/format_test/"
INPUT_DIR = os.path.join(MAIN_DIR, "data/input/")
OUTPUT_DIR = os.path.join(MAIN_DIR, "data/output/")
INPUT_DOCX = os.path.join(INPUT_DIR, "test_yaml.docx")
OUTPUT_DOCX = os.path.join(OUTPUT_DIR, "test_modified.docx")

STYLE_CONFIG_FILENAME = "style_config.yaml"
STYLE_SCHEMA_FILENAME = "style_config_schema.yaml"

In [3]:
doc = docx.Document(INPUT_DOCX)

In [4]:
formatter_config = DocumentFormatterConfig.load_and_validate_yaml(
    input_dir=INPUT_DIR,
    style_filename=STYLE_CONFIG_FILENAME,
    schema_filename=STYLE_SCHEMA_FILENAME
)

In [5]:
agent = DocumentFormattingAgent(doc, formatter_config)
agent.apply_all_styles()

In [6]:
doc.save(OUTPUT_DOCX)